# Combining Python, SQL DBs and AWS

## Aim: Create a RDB instance on AWS, connect to it through python, and execute queries.  


## Create an instance on AWS  
    
https://aws.amazon.com/getting-started/tutorials/create-mysql-db/

##  Using MYSQLWorkbench

Before we complicate things with using Python, we want to use MYSQLWorkbench to make sure we set up our AWS server correctly and can connect to it.  

- Create a connection to the AWS server
- Execute a simple create table query
- Drop the table


# Using Python with MYSQL DB

In [2]:
# make sure we have the package installed
!pip install mysql-connector-python

You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
## importing 'mysql.connector' 
import mysql.connector 


In [3]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector .connect(
    host = "localhost",
    user = "root",
    passwd = "dbms"
)

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost' (61)

### Documentation:

https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html

## Using a config File:

You do not want to make your credentials viewable to everyone who might see this file.  So instead of explicity stating your credentials, we want to import them from another file.  

In [4]:
import config

In [5]:
config.host

'kyledb.cyfsqcs7xvaz.us-east-2.rds.amazonaws.com'

In [25]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd
)

print(cnx) # it will print a connection object if everything is fine


In [26]:
cursor = cnx.cursor()

## Creating DB

In [27]:
import mysql.connector
from mysql.connector import errorcode

In [28]:
db_name = 'employees'

In [30]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

### Creating tables

In [31]:
DB_NAME = 'employees'

TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE employees ("
    "  emp_no int(11) NOT NULL AUTO_INCREMENT,"
    "  birth_date date NOT NULL,"
    "  first_name varchar(14) NOT NULL,"
    "  last_name varchar(16) NOT NULL,"
    "  gender enum('M','F') NOT NULL,"
    "  hire_date date NOT NULL,"
    "  PRIMARY KEY (emp_no)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE departments ("
    "  dept_no char(4) NOT NULL,"
    "  dept_name varchar(40) NOT NULL,"
    "  PRIMARY KEY (dept_no), UNIQUE KEY dept_name (dept_name)"
    ") ENGINE=InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE salaries ("
    "  emp_no int(11) NOT NULL,"
    "  salary int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT salaries_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE dept_emp ("
    "  emp_no int(11) NOT NULL,"
    "  dept_no char(4) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no), KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_emp_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_emp_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_manager'] = (
    "  CREATE TABLE dept_manager ("
    "  dept_no char(4) NOT NULL,"
    "  emp_no int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no),"
    "  KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_manager_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_manager_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['titles'] = (
    "CREATE TABLE titles ("
    "  emp_no int(11) NOT NULL,"
    "  title varchar(50) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date DEFAULT NULL,"
    "  PRIMARY KEY (emp_no,title,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT titles_ibfk_1 FOREIGN KEY (emp_no)"
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

In [13]:
TABLES['employees']

"CREATE TABLE employees (  emp_no int(11) NOT NULL AUTO_INCREMENT,  birth_date date NOT NULL,  first_name varchar(14) NOT NULL,  last_name varchar(16) NOT NULL,  gender enum('M','F') NOT NULL,  hire_date date NOT NULL,  PRIMARY KEY (emp_no)) ENGINE=InnoDB"

In [15]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

In [16]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table employees: OK
Creating table departments: OK
Creating table salaries: OK
Creating table dept_emp: OK
Creating table dept_manager: OK
Creating table titles: OK


## Inserting Data

In [32]:
from datetime import date, datetime, timedelta

#since we closed the connection, we need to reinstatiate it
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

tomorrow = datetime.now().date() + timedelta(days=1)

add_employee = ("INSERT INTO employees "
               "(first_name, last_name, hire_date, gender, birth_date) "
               "VALUES (%s, %s, %s, %s, %s)")
add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")

data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14))



In [33]:
# Insert new employee
cursor.execute(add_employee, data_employee)
emp_no = cursor.lastrowid

# Insert salary information
data_salary = {
  'emp_no': emp_no,
  'salary': 50000,
  'from_date': tomorrow,
  'to_date': date(9999, 1, 1),
}
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()

cursor.close() #closes cursor 
cnx.close() #closes connection

## Insert Many

cursor.executemany(operation, seq_of_params)

In [34]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

In [35]:
data = [
  ('Jane', date(2005, 2, 12)),
  ('Joe', date(2006, 5, 23)),
  ('John', date(2010, 10, 3)),
]
stmt = "INSERT INTO employees (first_name, hire_date) VALUES (%s, %s)"
cursor.executemany(stmt, data)

In [36]:
cnx.commit()

In [37]:
cursor.close()
cnx.close

<bound method CMySQLConnection.close of <mysql.connector.connection_cext.CMySQLConnection object at 0x10c6b0ba8>>

## Quyerying the DB

In [46]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

In [47]:

query = ("SELECT first_name, last_name, hire_date FROM employees "
         "WHERE hire_date BETWEEN %s AND %s")

hire_start = date(2005, 1, 1)
hire_end = date(2010, 12, 31)

cursor.execute(query, (hire_start, hire_end))

for (first_name, last_name, hire_date) in cursor:
  print("{}, {} was hired on {:%d %b %Y}".format(
    last_name, first_name, hire_date))

cursor.close()
cnx.close()

, Jane was hired on 12 Feb 2005
, Joe was hired on 23 May 2006
, John was hired on 03 Oct 2010



# Picking up from yesterday

## Select Statements

Select statements is the primary type of query you will write, as this is how we write q query to pull data back from a db


https://www.codecademy.com/articles/sql-commands

### SELECT
SELECT chooses the fields that you want displayed in your chart. This is the specific piece of information that you want to pull from your database.

**Syntax**

`SELECT first_name, last_name;`

Select * will pull back every column in the table


 ### FROM
FROM pinpoints the table that you want to pull the data from. 

**Syntax** 
`SELECT first_name, last_name
 FROM table;`

### WHERE
WHERE allows you to filter your query to be more specific. You  set up a conditional to filter your data

**syntax**

`SELECT first_name, last_name
FROM table
WHERE column = value`

### AND
AND allows you to add additional criteria to your WHERE statement. 


**Syntax**

`SELECT
     first_name, last_name
FROM
     table
WHERE
     column = value
AND
     column BETWEEN value AND value;`


### ORDER BY

Your ORDER BY clause will allow you to sort by any of the fields that you have specified in the SELECT statement. 

**Syntax**

`SELECT
     first_name, last_name
FROM
     table
WHERE
     column = value
AND
     column BETWEEN value AND value
ORDER BY
     column;`






### GROUP BY
"GROUP BY" is similar to the function in pandas where it will aggregate data that has similarities.

Here is your SQL query:

**Syntax**

`SELECT
     first_name, COUNT(last_name)
FROM
     table
WHERE
     column = value
AND
     column BETWEEN value AND value
GROUP BY
     column;`




### LIMIT
Depending on the amount of data you have in your database, it may take a long time to run the queries. It can be frustrating if you find yourself waiting a long time to run a query that you didn't really want to begin with. If you want to test our query, the LIMIT function is a great one to use because it allows you to limit the number of results you get.

**Syntax**

`SELECT
     first_name, COUNT(last_name)
FROM
     table
WHERE
     column = value
AND
     column BETWEEN value AND value
GROUP BY
     column
LIMIT
     100;`

## Applied: Working with our student data

### Import our student data

In [48]:
import json

f=open('students.json','r')
data=json.load(f)

### Create a connection and cursor to AWS db

In [49]:
# 
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

### Create a table for our student info

In [50]:

create_query = """
CREATE TABLE class_info (
name TEXT NOT NULL,
dob TEXT,
siblings REAL,
birthplace TEXT,
yearinnyc REAL,
favoritefood TEXT
);
"""

In [51]:
cursor.execute(create_query)

### Insert the student data into the table

In [5]:
#
keys_list = []
for student in data:
    print(student['name'])
    keys = student.keys()
    for word in keys:
        if word not in keys_list:
            keys_list.append(word)
print (keys_list)

Amora Sun
Alec Ryman
Catherine Wolk
Christopher Park
Eric Landstein
Jaden
James Moody
John Americk Canque
Justin Tennenbaum
Karolina Jozefowicz
Kathrin Verestoun
Kevin Sun
Kyle Baranko
mark cleverley
Matt Oliver
Michelle Venables
Paul E. Kruger
Quan Nguyen
Rebecca
Reuben Kavalov
Sarah Smith
Taylor Appel
['name', 'DOB', 'siblings', 'Birthplace', 'yearinnyc', 'favoritefood', 'birthdate', 'yearsinnyc', 'Favoritefood', 'birthplace']


In [1]:
students_list = []
    
for student in data:
    if 'DOB' in student.keys():
        student['birthdate'] = student['DOB']
    if 'Birthplace' in student.keys():
        student['birthplace'] = student.pop('Birthplace')
    if 'Favoritefood'in student.keys():
        student['favoritefood'] =student['favoritefood']

    student_tuple= (student['name'], student['birthdate'],
                   student['siblings'], student['birthplace'],
                   student['yearsinnyc'], student['favoritefood'])
    students_list.append(student_tuple)

NameError: name 'data' is not defined

### Write queries to answer the following questions:

**Questions**
- Which student was born closest to the cohort's graduation date?
- Which student has the most siblings?
- How many students are only children?
- Which 3 students have lived in NYC the shortest amount of time?
- How many students are native New Yorkers?
- Do any two students have the same favorite food?